<a href="https://colab.research.google.com/github/pamodaDilranga/SleepData_iOS_Ai/blob/main/SleepData_Fresh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [670]:
# from google.colab import files
# files.upload()

In [671]:
import tensorflow as tf

class callBack(tf.keras.callbacks.Callback):
  bestMse = [1000.0,1000.0]
  bestValMse = [1000.0,1000.0]
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_mse')<=bestValMse[1]):
      bestValMse[0] = logs.get('mse')
      bestValMse[1] = logs.get('val_mse')
    if(logs.get('mse')<=bestMse[0]):
      bestMse[0] = logs.get('mse')
      bestMse[1] = logs.get('val_mse')
    if(logs.get('mse')<=100.0)and(logs.get('val_mse')<=59.0):
      print("Reached Target")
      self.model.stop_training = True

  def on_train_end(self, logs={}):
    print('Best Val_MSE:',bestValMse)
    print('Best MSE:',bestMse)

In [672]:
callback = callBack()

In [673]:
bestMse = [1000.0,1000.0]
bestValMse = [1000.0,1000.0]

In [674]:
import pandas as pd
sleep_data = pd.read_csv('/content/sleepdata.csv', sep=';') 

In [675]:
sleep_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Start             887 non-null    object 
 1   End               887 non-null    object 
 2   Sleep quality     887 non-null    object 
 3   Time in bed       887 non-null    object 
 4   Wake up           246 non-null    object 
 5   Sleep Notes       652 non-null    object 
 6   Heart rate        162 non-null    float64
 7   Activity (steps)  887 non-null    int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 55.6+ KB


In [676]:
sleep_data['Heart rate'].fillna(sleep_data['Heart rate'].mean())

0      59.000000
1      72.000000
2      57.000000
3      60.604938
4      68.000000
         ...    
882    60.604938
883    60.604938
884    60.604938
885    60.604938
886    60.604938
Name: Heart rate, Length: 887, dtype: float64

In [677]:
sleep_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Start             887 non-null    object 
 1   End               887 non-null    object 
 2   Sleep quality     887 non-null    object 
 3   Time in bed       887 non-null    object 
 4   Wake up           246 non-null    object 
 5   Sleep Notes       652 non-null    object 
 6   Heart rate        162 non-null    float64
 7   Activity (steps)  887 non-null    int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 55.6+ KB


In [678]:
sleep_data['Sleep quality']=pd.Series(sleep_data['Sleep quality']).str.replace('%', '')
sleep_data['Sleep quality'] = pd.to_numeric(sleep_data['Sleep quality'], downcast='signed')

In [679]:
TimeInBed = []
for time in sleep_data['Time in bed']:
  minSec = time.split(':')
  timeInSec = (int(minSec[0])*60)+(int(minSec[1]))
  TimeInBed.append(timeInSec)

In [680]:
sleep_data['Time in bed'] = TimeInBed

In [681]:
sleep_data= sleep_data.dropna(axis=0)

**Handling Sleep Notes**

In [682]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [683]:
# sleepNoteArr = []
# for slpNote in sleep_data['Sleep Notes']:
#   if type(slpNote)==float:
#     sleepNoteArr.append('')
#   else:
#     sleepNoteArr.append(slpNote)
  
# sleep_data['Sleep Notes'] = sleepNoteArr

In [684]:
tokenizer = Tokenizer(num_words=100, split=' ', filters=':')
tokenizer.fit_on_texts(sleep_data['Sleep Notes'])

In [685]:
word_index = tokenizer.word_index
print(word_index)

{'drank': 1, 'coffee': 2, 'tea': 3, 'worked': 4, 'out': 5, 'stressful': 6, 'day': 7, 'ate': 8, 'late': 9}


In [686]:
train_sequences = tokenizer.texts_to_sequences(sleep_data['Sleep Notes'])

In [687]:
maxlen= max([len(x) for x in train_sequences])

In [688]:
train_padded = pad_sequences(train_sequences, padding='pre', truncating='pre', maxlen=maxlen)
print(train_padded)

[[0 0 0 ... 0 6 7]
 [0 0 0 ... 2 1 3]
 [0 0 0 ... 2 1 3]
 ...
 [0 0 1 ... 3 4 5]
 [0 0 0 ... 2 1 3]
 [0 0 0 ... 2 1 3]]


In [689]:
print(sleep_data['Sleep Notes'].shape)
print(train_padded.shape)

(155,)
(155, 8)


In [690]:
sleep_data['SNote1'] = 0
sleep_data['SNote2'] = 0
sleep_data['SNote3'] = 0
sleep_data['SNote4'] = 0
sleep_data['SNote5'] = 0
sleep_data['SNote6'] = 0
sleep_data['SNote7'] = 0
sleep_data['SNote8'] = 0

In [691]:
for idx, noteData in enumerate(train_padded):
  sleep_data['SNote1'][idx] = noteData[0]
  sleep_data['SNote2'][idx] = noteData[1]
  sleep_data['SNote3'][idx] = noteData[2]
  sleep_data['SNote4'][idx] = noteData[3]
  sleep_data['SNote5'][idx] = noteData[4]
  sleep_data['SNote6'][idx] = noteData[5]
  sleep_data['SNote7'][idx] = noteData[6]
  sleep_data['SNote8'][idx] = noteData[7]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

**Handling Wake Up**

In [692]:
# wakeupArr = []
# for wupNote in sleep_data['Sleep Notes']:
#   if type(wupNote)==float:
#     wakeupArr.append('')
#   else:
#     wakeupArr.append(wupNote)
  
# sleep_data['Wake up'] = wakeupArr

In [693]:
tokenizer = Tokenizer(num_words=100, split=' ', filters='')
tokenizer.fit_on_texts(sleep_data['Wake up'])

In [694]:
word_index = tokenizer.word_index
print(word_index)

{':)': 1, ':|': 2}


In [695]:
train_sequences = tokenizer.texts_to_sequences(sleep_data['Wake up'])

In [696]:
maxlen= max([len(x) for x in train_sequences])

In [697]:
train_padded = pad_sequences(train_sequences, padding='pre', truncating='pre', maxlen=maxlen)

In [698]:
print(sleep_data['Wake up'].shape)
print(train_padded.shape)

(155,)
(155, 1)


In [699]:
sleep_data['WakeUp'] = 0

In [700]:
for idx, noteData in enumerate(train_padded):
  sleep_data['WakeUp'][idx] = noteData[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**End Of Handling Wake Up**

In [701]:
sleep_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155 entries, 1 to 241
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Start             155 non-null    object 
 1   End               155 non-null    object 
 2   Sleep quality     155 non-null    int8   
 3   Time in bed       155 non-null    int64  
 4   Wake up           155 non-null    object 
 5   Sleep Notes       155 non-null    object 
 6   Heart rate        155 non-null    float64
 7   Activity (steps)  155 non-null    int64  
 8   SNote1            155 non-null    int64  
 9   SNote2            155 non-null    int64  
 10  SNote3            155 non-null    int64  
 11  SNote4            155 non-null    int64  
 12  SNote5            155 non-null    int64  
 13  SNote6            155 non-null    int64  
 14  SNote7            155 non-null    int64  
 15  SNote8            155 non-null    int64  
 16  WakeUp            155 non-null    int64  
dt

In [702]:
sleep_data.drop('Start', inplace=True, axis=1)
sleep_data.drop('End', inplace=True, axis=1)
sleep_data.drop('Wake up', inplace=True, axis=1)
sleep_data.drop('Sleep Notes', inplace=True, axis=1)
# sleep_data.drop('WakeUp', inplace=True, axis=1)

In [703]:
sleep_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155 entries, 1 to 241
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sleep quality     155 non-null    int8   
 1   Time in bed       155 non-null    int64  
 2   Heart rate        155 non-null    float64
 3   Activity (steps)  155 non-null    int64  
 4   SNote1            155 non-null    int64  
 5   SNote2            155 non-null    int64  
 6   SNote3            155 non-null    int64  
 7   SNote4            155 non-null    int64  
 8   SNote5            155 non-null    int64  
 9   SNote6            155 non-null    int64  
 10  SNote7            155 non-null    int64  
 11  SNote8            155 non-null    int64  
 12  WakeUp            155 non-null    int64  
dtypes: float64(1), int64(11), int8(1)
memory usage: 20.9 KB


In [704]:
X = sleep_data.loc[:, sleep_data.columns != 'Sleep quality']
Y = sleep_data.loc[:, sleep_data.columns == 'Sleep quality']

In [705]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [706]:
model = Sequential([
     Dense(units=64, activation='relu'),
     Dense(units=32, activation='relu'),
     Dense(units=1)
])

In [707]:
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['mse'])

In [708]:
model.fit(x=X,y=Y, epochs=250, validation_split=(0.2), callbacks=[callback])

Epoch 1/250

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

4/4 [==============================] - 0s 33ms/step - loss: 3678.0798 - mse: 3678.0798 - val_loss: 1189.1116 - val_mse: 1189.1116
Epoch 2/250
4/4 [==============================] - 0s 5ms/step - loss: 657.9722 - mse: 657.9722 - val_loss: 64.4354 - val_mse: 64.4354
Epoch 3/250
4/4 [==============================] - 0s 5ms/step - loss: 147.7550 - mse: 147.7550 - val_loss: 320.9286 - val_mse: 320.9286
Epoch 4/250
4/4 [==============================] - 0s 5ms/step - loss: 444.0388 - mse: 444.038

In [709]:
#loss: 105.1226 - mse: 105.1226 - val_loss: 65.1121 - val_mse: 65.1121 with activity
#loss: 102.7348 - mse: 102.7348 - val_loss: 62.5017 - val_mse: 62.5017 without sleep  notes
#loss: 102.5009 - mse: 102.5009 - val_loss: 62.1021 - val_mse: 62.1021 with sleep notes
#loss: 102.3190 - mse: 102.3190 - val_loss: 62.3128 - val_mse: 62.3128 with sleep notes and wake up
#loss: 101.6704 - mse: 101.6704 - val_loss: 59.8196 - val_mse: 59.8196 50 epochs
#loss:  99.3130 - mse:  99.3130 - val_loss: 65.5199 - val_mse: 65.5199 with heart rate filled with fillna(mean)
#loss: 101.6605 - mse: 101.6605 - val_loss: 59.4113 - val_mse: 59.4113 With Call back mse: 101.6704 val_mse: 59.8196
#loss: 100.7489 - mse: 100.7489 - val_loss: 61.2192 - val_mse: 61.2192 with every attribute exept start and end with out Call back
#loss: 92.3622  - mse: 92.3622  - val_loss: 190.7880- val_mse: 190.7880 With 162 values filling sleep notes for na values
#loss: 91.5628  - mse: 91.5628  - val_loss: 125.4840- val_mse: 125.4840 With pre procesing WakeUp in order to avoid dropna wake up

In [710]:
# Best Val_MSE: [105.44902038574219, 58.143531799316406]
# Best MSE: [99.47660064697266, 68.79240417480469]

In [713]:
model.save('SleepData_bestYet_model.h5')

In [712]:
sleep_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155 entries, 1 to 241
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sleep quality     155 non-null    int8   
 1   Time in bed       155 non-null    int64  
 2   Heart rate        155 non-null    float64
 3   Activity (steps)  155 non-null    int64  
 4   SNote1            155 non-null    int64  
 5   SNote2            155 non-null    int64  
 6   SNote3            155 non-null    int64  
 7   SNote4            155 non-null    int64  
 8   SNote5            155 non-null    int64  
 9   SNote6            155 non-null    int64  
 10  SNote7            155 non-null    int64  
 11  SNote8            155 non-null    int64  
 12  WakeUp            155 non-null    int64  
dtypes: float64(1), int64(11), int8(1)
memory usage: 20.9 KB
